In [4]:
torch.cuda.is_available()

True

In [3]:
!nvidia-smi

Tue Oct 26 17:32:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 465.19.01    CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| 28%   37C    P8    16W / 260W |    332MiB / 11016MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:02:00.0 Off |                  N/A |
| 35%   

In [33]:
import torch
import os
torch.cuda.empty_cache()
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
torch.cuda.synchronize()

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [32]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
print(torch.cuda.memory_allocated())

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
print(torch.cuda.memory_allocated())

torch.cuda.synchronize()
print(torch.cuda.memory_allocated())

0
0


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [31]:
#some Python subprocesses are still alive. You may find them via
!ps -elf | grep python 

4 S root      1057     1  0  80   0 - 42635 -      12:48 ?        00:00:00 /usr/bin/python3 /usr/bin/networkd-dispatcher --run-startup-triggers
4 S root      1240     1  0  80   0 - 46814 -      12:48 ?        00:00:00 /usr/bin/python3 /usr/share/unattended-upgrades/unattended-upgrade-shutdown --wait-for-signal
0 S mostafa   3737  3713  0  80   0 - 61836 ep_pol 12:52 pts/0    00:00:08 /home/mostafa/Marwa/Environments/nicMSlesion_env/bin/python /home/mostafa/Marwa/Environments/nicMSlesion_env/bin/jupyter-notebook
0 S mostafa   3846  3737  0  80   0 - 190241 ep_pol 12:52 ?       00:00:00 /home/mostafa/Marwa/Environments/nicMSlesion_env/bin/python -m ipykernel_launcher -f /home/mostafa/.local/share/jupyter/runtime/kernel-667ab459-776d-4525-9e52-7d661c8fd42d.json
0 S mostafa   4010  3737  0  80   0 - 4201853 ep_pol 12:53 ?      00:00:05 /home/mostafa/Marwa/Environments/nicMSlesion_env/bin/python -m ipykernel_launcher -f /home/mostafa/.local/share/jupyter/runtime/kernel-f0b43fe9-59e1-4943-9

In [30]:
!kill -9 1240

/bin/bash: line 0: kill: (1240) - Operation not permitted


In [21]:
import torch
import os

print('cuda available? ', torch.cuda.is_available())
print('cuda count ', torch.cuda.device_count())
print('current device ',torch.cuda.current_device())

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print('current device ',torch.cuda.current_device())

torch.cuda.set_device(1)
print('current device ',torch.cuda.current_device())

cuda available?  True
cuda count  2
current device  1
current device  1
current device  1


In [5]:
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_name(1))

NVIDIA GeForce RTX 2080 Ti
NVIDIA GeForce RTX 2080 Ti


In [23]:
!nvidia-smi

Tue Oct 26 15:27:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 465.19.01    CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| 28%   35C    P8    11W / 260W |  10975MiB / 11016MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:02:00.0 Off |                  N/A |
| 27%   

In [7]:
watch -n 2 nvidia-smi

SyntaxError: invalid syntax (2578344406.py, line 1)

In [15]:
torch.cuda.clear_memory_allocated()

AttributeError: module 'torch.cuda' has no attribute 'clear_memory_allocated'

In [1]:
from dataset_loader import *
from config.settings import *
import dataset_loader as dl

In [2]:
st = Settings()
options = st.get_options()

In [44]:
import torch
torch.__version__

'1.9.1'

In [45]:
import torch.nn as nn
nn.AvgPool3d?

In [4]:
len_source_loader = len(source_loader)
iter_source = iter(source_loader)
num_iter = len_source_loader
for i in range(1, num_iter):
    data_source, label_source = iter_source.next()
    

NameError: name 'source_loader' is not defined

In [7]:
x= enumerate(data_loader)

In [6]:
from utils.data_preprocess import *
df = generate_csv(options)
split_folds(options['train_csv_path'], options['seed'], options['k_fold'])

/home/microsoft/Medical_Analysis/Code/Environments/medical_pytorch_env/lib/python3.7/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


In [22]:
df.columns

Index(['root_path', 'patient_id', 'study', 'lesion', 'FLAIR', 'T1', 'fold'], dtype='object')

In [37]:
import pandas as pd



(array([['training01', '_01'],
        ['training02', '_04'],
        ['training03', '_02'],
        ['training04', '_02'],
        ['training05', '_02']], dtype=object),
 ['training01_01',
  'training02_04',
  'training03_02',
  'training04_02',
  'training05_02'])

In [12]:
scan_list

['training01_01',
 'training01_02',
 'training01_03',
 'training01_04',
 'training02_01',
 'training02_02',
 'training02_03',
 'training02_04',
 'training03_01',
 'training03_02',
 'training03_03',
 'training03_04',
 'training03_05',
 'training04_01',
 'training04_02',
 'training04_03',
 'training04_04',
 'training05_01',
 'training05_02',
 'training05_03',
 'training05_04']